In [1]:
from py2neo import Graph
from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt
import neo4jupyter

neo_host = "neo4j"
neo_port = 7687

graph = Graph(f"bolt://{neo_host}:{neo_port}")
driver = GraphDatabase.driver(f"bolt://{neo_host}:{neo_port}")

In [ ]:
NB_SELECTED = 4

graph.run("""MATCH ()-[r:Dissed]->(target)
                RETURN target, count(r) AS num
                ORDER BY num DESC
                LIMIT $nb_selected
            """, parameters={"nb_selected": NB_SELECTED}).to_data_frame()[['target']]

In [ ]:
NB_SELECTED = 4

graph.run("""MATCH (artist)-[r:Dissed]->()
                RETURN artist, count(r) AS num
                ORDER BY num DESC
                LIMIT $nb_selected
            """, parameters={"nb_selected": NB_SELECTED}).to_data_frame()[['artist']]

In [ ]:
neo4jupyter.init_notebook_mode()
neo4jupyter.draw(graph, options={"Artist": "name", "Target": "name"})
#sub_g = graph.run("""
#    MATCH (artist)-[r:Dissed]->(target)
#    RETURN *""").to_subgraph()
#neo4jupyter.draw_subgraph(sub_g, options={"artist": "wikidata_id", "target": "name"})

In [ ]:
%matplotlib inline

results = driver.session().run("""
MATCH (n)-[r]->(c) RETURN *
""")

G = nx.MultiDiGraph()

for node in results.graph()._nodes.values():
    G.add_node(node.id, labels=node._labels, properties=node._properties)

rels = list(results.graph()._relationships.values())
for rel in rels:
    G.add_edge(rel.start_node.id, rel.end_node.id, key=rel.id, type=rel.type, properties=rel._properties)

nx.draw(G, node_size=10, arrowsize=2)
plt.show()